In [138]:
from sympy import FiniteSet, EmptySet, Complement

class BinaryRelation:
    
    def __init__(self,relation=None,domain=None,codomain=None):      
        self._relation = FiniteSet(EmptySet) if relation is None else FiniteSet(*relation)
        self._domain = FiniteSet(EmptySet) if domain is None else FiniteSet(*domain)
        self._codomain = FiniteSet(EmptySet) if codomain is None else FiniteSet(*codomain)
        
    def __repr__(self):
        relation = self.get_relation()
        if len(relation) == 0:
            return "{}"
        r = "{"
        for e in relation:
            r += f"{str(e)},"
        return r[:-1] + "}" 
    
    def __eq__(self, binary_relation):
        return self.get_relation() == binary_relation.get_relation()
    
    def __ne__(self, binary_relation):
        return self.get_relation() != binary_relation.get_relation()
    
    def __lt__(self, binary_relation):
        return self.get_relation() < binary_relation.get_relation()
        
    def __le__(self, binary_relation):
        return self.get_relation() <= binary_relation.get_relation()
    
    def __gt__(self, binary_relation):
        return self.get_relation() > binary_relation.get_relation()
        
    def __ge__(self, binary_relation):
        return self.get_relation() >= binary_relation.get_relation()  
        
    def set_domain(self, domain):
        self._domain = FiniteSet(*domain)
        
    def get_domain(self):
        return self._domain
    
    def set_codomain(self, codomain):
        self._codomain = FiniteSet(*codomain)
        
    def get_codomain(self):
        return self._codomain
        
    def set_relation(self, relation):
        self._relation = FiniteSet(*relation)
        
    def get_relation(self):
        return self._relation
    
    def dom(self):
        a = set()
        relation = self.get_relation();
        for x,y in relation:
            a.add(x)
        return FiniteSet(*a)
        
    def rng(self):
        a = set()
        relation = self.get_relation();
        for x,y in relation:
            a.add(y)
        return FiniteSet(*a)
    
    def union(self, binary_relation):
        u = self.get_relation().union(binary_relation.get_relation())
        return BinaryRelation(u,self.get_domain(), self.get_codomain())
        
    def intersection(self, binary_relation):
        intersec = self.get_relation().intersection(binary_relation.get_relation())
        return BinaryRelation(intersec,self.get_domain(), self.get_codomain())
        
    def difference(self, binary_relation):
        diff = set()
        relation1 = self.get_relation()
        relation2 = binary_relation.get_relation()
        for x in relation1:
            if x not in relation2:
                diff.add(x)
        return BinaryRelation(diff,self.get_domain(), self.get_codomain())
        
    def complement(self):
        comp = Complement(self.cartesian_product(), self.get_relation())
        return BinaryRelation(comp,self.get_domain(),self.get_codomain())
    
    def cartesian_product(self):
        product = self.get_domain() * self.get_codomain()
        return FiniteSet(*product)
    
    def composition(self, binary_relation):

        comp = set()
        relation1 = self.get_relation()
        relation2 = binary_relation.get_relation()
        intersection = self.get_codomain().intersection(binary_relation.get_domain())
        
        for x1,y1 in relation1:
            if y1 in intersection:
                for x2,y2 in relation2:
                    if y1 == x2:
                        comp.add((x1,y2))
                        
        return BinaryRelation(comp,self.get_domain(),binary_relation.get_codomain())
    
    def converse(self):
        
        conv = set()
        relation = self.get_relation()
        
        for x,y in relation:
            conv.add((y,x))
            
        return BinaryRelation(conv, self.get_codomain(), self.get_domain())
    
    def image(self, a):
        img = FiniteSet()
        for x in a:
            img = img.union(self.in_relation_with(x))
        return img
    
    def preimage(self, a):
        preimg = FiniteSet()
        conv = self.converse()
        for x in a:
            preimg = preimg.union(conv.in_relation_with(x))
        return preimg
    
    def in_relation_with(self, variable):
        
        a = set()
        relation = self.get_relation()
        
        for x,y in relation:
            if x == variable:
                a.add(y)
                
        return FiniteSet(*a)
    
    def is_binary_relation(self):
        return self.get_relation() <= self.cartesian_product()
    
    def is_homogeneous_relation(self):
        return self.is_binary_relation() and self.get_domain() == self.get_codomain()
    
    def is_identity_relation(self):
        return self.is_homogeneous_relation() and all (list (map (lambda xy: xy[0] == xy[1], self.get_relation())))
    
    


In [139]:
r = BinaryRelation({(1,1),(1,2),(2,1),(3,3)},{1,2,3},{1,2,3})
s = BinaryRelation({(1,3),(2,2),(3,1)},{1,2,3},{1,2,3})
rs = r.composition(s)

print(f"R = {r}")
print(f"S = {s}")
print(f"R∘S = {rs}")

print()

print(f"is binary relation? R = {r.is_binary_relation()}")
print(f"is binary relation? S = {s.is_binary_relation()}")

print()

print(f"is homogeneous relation? R = {r.is_homogeneous_relation()}")
print(f"is homogeneous relation? S = {s.is_homogeneous_relation()}")

print()

print(f"is identity relation? R = {r.is_identity_relation()}")
print(f"is identity relation? S = {s.is_identity_relation()}")

print()

print(f"S[A] = {s.image({1,2})}")
print(f"S-1[A] = {s.preimage({1,2})}")

print()

print(f"dom S = {s.dom()}")
print(f"rng S = {s.rng()}")

print()

print(f"R == S: {r == s}")
print(f"R != S: {r != s}")
print(f"R < S: {r < s}")
print(f"R <= S: {r <= s}")
print(f"R > S: {r > s}")
print(f"R >= S: {r >= s}")

print()

print(f"R union S = {r.union(s)}")
print(f"R intersection S = {r.intersection(s)}")
print(f"R difference S = {r.difference(s)}")

print()

print(f"R complement = {r.complement()}")
print(f"R∘S converse = {rs.converse()}")

print()

print(f"'1' is in relation with: {r.in_relation_with(1)} in R")

R = {(1, 1),(1, 2),(2, 1),(3, 3)}
S = {(1, 3),(2, 2),(3, 1)}
R∘S = {(1, 2),(1, 3),(2, 3),(3, 1)}

is binary relation? R = True
is binary relation? S = True

is homogeneous relation? R = True
is homogeneous relation? S = True

is identity relation? R = False
is identity relation? S = False

S[A] = FiniteSet(2, 3)
S-1[A] = FiniteSet(2, 3)

dom S = FiniteSet(1, 2, 3)
rng S = FiniteSet(1, 2, 3)

R == S: False
R != S: True
R < S: False
R <= S: False
R > S: False
R >= S: False

R union S = {(1, 1),(1, 2),(1, 3),(2, 1),(2, 2),(3, 1),(3, 3)}
R intersection S = {}
R difference S = {(1, 1),(1, 2),(2, 1),(3, 3)}

R complement = {(1, 3),(2, 2),(2, 3),(3, 1),(3, 2)}
R∘S converse = {(1, 3),(2, 1),(3, 1),(3, 2)}

'1' is in relation with: FiniteSet(1, 2) in R
